# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
import pandas as pd
import numpy as np
import re
from scipy.stats import ttest_ind
pd.options.display.max_rows = 100

In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {
    'OH': 'Ohio', 
    'KY': 'Kentucky', 
    'AS': 'American Samoa', 
    'NV': 'Nevada', 'WY': 'Wyoming', 
    'NA': 'National', 
    'AL': 'Alabama', 
    'MD': 'Maryland', 
    'AK': 'Alaska', 
    'UT': 'Utah', 
    'OR': 'Oregon', 
    'MT': 'Montana', 
    'IL': 'Illinois', 
    'TN': 'Tennessee', 
    'DC': 'District of Columbia', 
    'VT': 'Vermont', 
    'ID': 'Idaho', 
    'AR': 'Arkansas', 
    'ME': 'Maine', 
    'WA': 'Washington', 
    'HI': 'Hawaii', 
    'WI': 'Wisconsin', 
    'MI': 'Michigan', 
    'IN': 'Indiana', 
    'NJ': 'New Jersey', 
    'AZ': 'Arizona', 
    'GU': 'Guam', 
    'MS': 'Mississippi', 
    'PR': 'Puerto Rico', 
    'NC': 'North Carolina', 
    'TX': 'Texas', 
    'SD': 'South Dakota', 
    'MP': 'Northern Mariana Islands', 
    'IA': 'Iowa', 'MO': 'Missouri', 
    'CT': 'Connecticut', 
    'WV': 'West Virginia', 
    'SC': 'South Carolina', 
    'LA': 'Louisiana', 'KS': 'Kansas', 
    'NY': 'New York', 'NE': 'Nebraska', 
    'OK': 'Oklahoma', 'FL': 'Florida', 
    'CA': 'California', 'CO': 'Colorado', 
    'PA': 'Pennsylvania', 'DE': 'Delaware', 
    'NM': 'New Mexico', 'RI': 'Rhode Island', 
    'MN': 'Minnesota', 'VI': 'Virgin Islands', 
    'NH': 'New Hampshire', 'MA': 'Massachusetts', 
    'GA': 'Georgia', 
    'ND': 'North Dakota', 
    'VA': 'Virginia'}

In [236]:
def get_list_of_university_towns():
#    '''Returns a DataFrame of towns and the states they are in from the 
#    university_towns.txt list. The format of the DataFrame should be:
#    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
#    columns=["State", "RegionName"]  )
#    
#    The following cleaning needs to be done:
#
#    1. For "State", removing characters from "[" to the end.
#    2. For "RegionName", when applicable, removing every character from " (" to the end.
#    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
#    
    unitown = pd.read_csv('university_towns.txt', sep='\n', header=None)
    unitown.loc[:, 'State'] = np.NaN
    states = unitown.loc[unitown.loc[:, 0].str.contains('edit')].loc[:, 0]
    states = states.str.split('[', expand=True)[0]
    unitown.loc[states.index, 'State'] = states.values
    unitown.loc[:, 'State'].fillna(method='ffill', inplace=True)
    unitown.drop(states.index, inplace=True)
    unitown.loc[:, 'RegionName'] = unitown.loc[:, 0].apply(lambda x: x.split('(')[0].strip())
    unitown.drop(0, axis=1, inplace=True)
    return unitown

get_list_of_university_towns()

,State,RegionName
1,Alabama,Auburn
2,Alabama,Florence
3,Alabama,Jacksonville
4,Alabama,Livingston
5,Alabama,Montevallo
...,...,...
561,Wisconsin,River Falls
562,Wisconsin,Stevens Point
563,Wisconsin,Waukesha
564,Wisconsin,Whitewater


In [74]:
def load_data():
    recession_df = pd.read_excel('gdplev.xls', usecols=[4, 6], skiprows=7)
    recession_df.columns = ['Quarter', 'GDP']
    recession_df.loc[
        recession_df.iloc[1:].loc[recession_df.iloc[1:, 1].values > recession_df.iloc[:-1, 1].values].index,
        'Growth/Decline'
    ] = 'Growth'
    recession_df.loc[1:, 'Growth/Decline'].fillna('Decline', inplace=True)
    recession_df = recession_df[recession_df.iloc[:, 0].str.startswith('20')]
    return recession_df

def get_recession_start():
    #'''Returns the year and quarter of the recession start time as a 
    #string value in a format such as 2005q3'''
    #A recession is defined as starting with two consecutive quarters of GDP decline,
    #and ending with two consecutive quarters of GDP growth.
    recession_start = load_data()
    recession_start = recession_start[
        (recession_start.loc[:, 'Growth/Decline'].shift(-1) == 'Decline') & 
        (recession_start.loc[:, 'Growth/Decline'] == 'Decline')].loc[:, 'Quarter']
    return recession_start.iloc[0]
get_recession_start()

'2008q3'

In [75]:
def get_recession_end():
#    '''Returns the year and quarter of the recession end time as a 
#    string value in a format such as 2005q3'''
    recession_end = load_data()
    recession_start = recession_end.loc[recession_end.loc[:, 'Quarter'] == get_recession_start()].copy().index.values.item()
    recession_end = recession_end.loc[recession_start:].copy()
    recession_end = recession_end[        
        (recession_end.loc[:, 'Growth/Decline'].shift(-1) == 'Growth') &
        (recession_end.loc[:, 'Growth/Decline'] == 'Growth')
    ].loc[:, 'Quarter'].iloc[1]
    return recession_end
get_recession_end()

'2009q4'

In [110]:
def get_recession_bottom():
#    '''Returns the year and quarter of the recession bottom time as a 
#    string value in a format such as 2005q3'''
    recession_bottom = load_data()
    recession_start = get_recession_start()
    recession_start = recession_bottom[recession_bottom.loc[:, 'Quarter'] == recession_start].index.values.item()
    recession_end = get_recession_end()
    recession_end = recession_bottom[recession_bottom.loc[:, 'Quarter'] == recession_end].index.values.item()
    recession_bottom = recession_bottom.loc[recession_start:recession_end]
    recession_bottom = recession_bottom[recession_bottom['GDP'] == recession_bottom.loc[:, 'GDP'].min()].copy()
    recession_bottom = recession_bottom.loc[:, 'Quarter'].values.item()
    return recession_bottom

get_recession_bottom()

'2009q2'

In [185]:
def load_housing():
    housing_df = pd.read_csv('City_Zhvi_AllHomes.csv')
    
    return housing_df

def convert_housing_data_to_quarters():
#    '''Converts the housing data to quarters and returns it as mean 
#    values in a dataframe. This dataframe should be a dataframe with
#    columns for 2000q1 through 2016q3, and should have a multi-index
#    in the shape of ["State","RegionName"].
#    
#    Note: Quarters are defined in the assignment description, they are
#    not arbitrary three month periods. A quarter is a specific three month period, 
#    Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
#    
#    The resulting dataframe should have 67 columns, and 10,730 rows.
#    '''
    housing_df = load_housing()
    housing_df.loc[:, 'State'] = housing_df.loc[:, 'State'].apply(lambda x: states[x])
    housing_df.set_index(['State', 'RegionName'], inplace=True)
    housing_df.sort_index(inplace=True)
    housing_df = housing_df.loc[:, housing_df.columns[housing_df.columns.str.startswith('20')]].copy()
    housing_df.columns = pd.to_datetime(housing_df.columns)
    housing_df = housing_df.resample('Q', axis=1).mean()
    housing_df.rename(columns = lambda x: str(x.to_period('Q')).lower(), inplace=True)
    return housing_df

convert_housing_data_to_quarters()

2000q1         2000q2         2000q3  \
State   RegionName                                                 
Alabama Adamsville    69033.333333   69166.666667   69800.000000   
        Alabaster    122133.333333  123066.666667  123166.666667   
        Albertville   73966.666667   72600.000000   72833.333333   
        Arab          83766.666667   81566.666667   81333.333333   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        101533.333333  104566.666667  108366.666667   
        Casper        89233.333333   89600.000000   89733.333333   
        Cheyenne     116866.666667  120033.333333  121533.333333   
        Evansville   128033.333333  128766.666667  130833.333333   
        Pine Bluffs   93733.333333   95066.666667   94633.333333   

                            2000q4         2001q1         2001q2  \
State   RegionName                                                 
Alabama Adamsville    71966.666667   73466.666667   74000.000000   
        Alabaster    123700.000000  123233.333333  125133.333333   
        Albertville   74200.000000   75900.000000   76000.000000   
        Arab          82966.666667   84200.000000   84533.333333   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        113000.000000  115833.333333  117200.000000   
        Casper        93166.666667   95500.000000   97633.333333   
        Cheyenne     123633.333333  125533.333333  126300.000000   
        Evansville   132066.666667  130566.666667  131433.333333   
        Pine Bluffs   98066.666667  103233.333333  104600.000000   

                            2001q3         2001q4         2002q1  \
State   RegionName                                                 
Alabama Adamsville    73333.333333   73100.000000   73333.333333   
        Alabaster    127766.666667  127200.000000  127300.000000   
        Albertville   72066.666667   73566.666667   76533.333333   
        Arab          81666.666667   83900.000000   87266.666667   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        117800.000000  117633.333333  117333.333333   
        Casper        99433.333333  100633.333333  101733.333333   
        Cheyenne     126466.666667  128133.333333  128466.666667   
        Evansville   132400.000000  133466.666667  133300.000000   
        Pine Bluffs  106500.000000  104066.666667  102233.333333   

                            2002q2  ...         2014q2         2014q3  \
State   RegionName                  ...                                 
Alabama Adamsville    73133.333333  ...   77066.666667   75966.666667   
        Alabaster    128000.000000  ...  147133.333333  147633.333333   
        Albertville   76366.666667  ...   84033.333333   84766.666667   
        Arab          87700.000000  ...  113366.666667  111700.000000   
        Ardmore                NaN  ...  140533.333333  139566.666667   
...                            ...  ...            ...            ...   
Wyoming Burns        117233.333333  ...  168866.666667  161933.333333   
        Casper       101533.333333  ...  175766.666667  177300.000000   
        Cheyenne     129633.333333  ...  177466.666667  176733.333333   
        Evansville   131066.666667  ...  296733.333333  305666.666667   
        Pine Bluffs  103566.666667  ...  148666.666667  154366.666667   

                            2014q4         2015q1         2015q2  \
State   RegionName                                                 
Alabama Adamsville    71900.000000   71666.666667   73033.333333   
        Alabaster    148700.000000  148900.000000  149566.666667   
        Albertville   86800.000000   88466.666667   89500.000000   
        Arab         111600.000000  110166.666667  109433.333333   
        Ardmore

In [281]:
def run_ttest():
#    '''First creates new data showing the decline or growth of housing prices
#    between the recession start and the recession bottom. Then runs a ttest
#    comparing the university town values to the non-university towns values, 
#    return whether the alternative hypothesis (that the two groups are the same)
#    is true or not as well as the p-value of the confidence. 
#    
#    Return the tuple (different, p, better) where different=True if the t-test is
#    True at a p<0.01 (we reject the null hypothesis), or different=False if 
#    otherwise (we cannot reject the null hypothesis). The variable p should
#    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
#    value for better should be either "university town" or "non-university town"
#    depending on which has a lower mean price ratio (which is equivilent to a
#    reduced market loss).'''

    recession_start = get_recession_start()
    recession_bottom = get_recession_bottom()
    unitown = get_list_of_university_towns()
    unitown.reset_index(inplace=True)
    housing_df = convert_housing_data_to_quarters()
    quarter_before = housing_df.columns[housing_df.columns.get_loc(recession_start) - 1]
    housing_df = housing_df.loc[:, quarter_before:recession_bottom].copy()
    housing_df['Price Ratio'] = housing_df.iloc[:, -1] / housing_df.iloc[:, 0]
    housing_df.reset_index(inplace=True)
    unitown_df = pd.merge(housing_df, unitown, how='inner', left_on='RegionName', right_on='RegionName')
    unitown_df.drop(['index', 'State_y'], axis=1, inplace=True)
    unitown_df.rename({'State_x':'State'}, axis=1, inplace=True)
    unitown_df.set_index('RegionName', inplace=True)
    housing_df.set_index('RegionName', inplace=True)
    othertown_df = housing_df.loc[~housing_df.index.isin(unitown_df.index)].copy()
    t_stat, p_value = ttest_ind(unitown_df.loc[:, 'Price Ratio'], othertown_df.loc[:, 'Price Ratio'], nan_policy='omit')
    return (
        f'T-Statistic = {t_stat}',
        f'P-value = {p_value}',
        f'Different = {p_value<0.01}',
        f"Better = {'University town' if unitown_df.loc[:, 'Price Ratio'].mean() > othertown_df.loc[:, 'Price Ratio'].mean() else 'Non-University Town'}",
        f"Mean university town property price ratio = {unitown_df.loc[:, 'Price Ratio'].mean()}",
        f"Mean non-university town property price ratio = {othertown_df.loc[:, 'Price Ratio'].mean()}",
    )

run_ttest()

('T-Statistic = 4.892899310169373',
 'P-value = 1.009088013905517e-06',
 'Different = True',
 'Better = University town',
 'Mean university town property price ratio = 0.9525854011772523',
 'Mean non-university town property price ratio = 0.9381581822815416')

In [221]:
pd.merge?